# STS Benchmark Datasets

For more information, see the implementation of https://github.com/kawine/usif

In [19]:
%load_ext autotime
import numpy as np
import logging
import pandas as pd
import pathlib

from gensim.models import Word2Vec, FastText
from gensim.models.keyedvectors import KeyedVectors, FastTextKeyedVectors

from fse.models.average import Average
from fse.models.sif import SIF
from fse.models.inputs import IndexedSentence

from re import sub

from scipy.stats import pearsonr

logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s',
                    level=logging.INFO
                   )

file_path = "data/stsbenchmark/sts-test.csv"

def normalize_text(sentence):
    return [sub("[^a-zA-Z]", "", w.lower()) for w in sentence.split()] 

def compute_similarities(task_length, model):
    sims = []
    for i, j in zip(range(task_length), range(task_length, 2*task_length)):
        sims.append(model.sv.similarity(i,j))
    return sims

sts_data = pd.read_csv(file_path, sep=",", error_bad_lines=True, names=["cpt","1","task","3","sim","sent_a","sent_b"])
sts_data.dropna(inplace=True)
sts_data.sent_a = sts_data.sent_a.apply(normalize_text)
sts_data.sent_b = sts_data.sent_b.apply(normalize_text)
sts_data.task = sts_data.task.str.replace("2012test", "2012")
sts_data.task = sts_data.task.str.replace("2012train", "None")
sts_data = sts_data[sts_data.task != "None"]

tasks = sorted(["2012", "2013", "2014", "2015", "2016", "2017"])

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 88.4 ms


In [20]:
w2v = KeyedVectors.load("/Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model")
ft = FastTextKeyedVectors.load("/Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.model")

2019-08-19 20:46:41,037 : MainThread : INFO : loading Word2VecKeyedVectors object from /Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model
2019-08-19 20:46:47,952 : MainThread : INFO : loading vectors from /Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model.vectors.npy with mmap=None
2019-08-19 20:46:52,990 : MainThread : INFO : setting ignored attribute vectors_norm to None
2019-08-19 20:46:53,000 : MainThread : INFO : loaded /Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model
2019-08-19 20:46:53,002 : MainThread : INFO : loading FastTextKeyedVectors object from /Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.model
2019-08-19 20:47:01,238 : MainThread : INFO : loading vectors from /Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.model.vectors.npy with mmap=None
2019-08-19 20:47:04,638 : MainThread : INFO : loading vectors_vocab from /Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.mo

time: 31.9 s


In [21]:
w2v_avg = Average(w2v, wv_mapfile_path = "data/vectors/w2v", lang_freq="en")
ft_avg = Average(ft, wv_mapfile_path = "data/vectors/ft", lang_freq="en")

/Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/models/base_s2v.py:157: UserWarning: C extension not loaded, training/inferring will be slow. Install a C compiler and reinstall fse.
  "C extension not loaded, training/inferring will be slow. "
2019-08-19 20:47:13,036 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/w2v
2019-08-19 20:47:14,178 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en
2019-08-19 20:47:20,020 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/ft
2019-08-19 20:47:20,543 : MainThread : INFO : loading pre-existing vocab from data/vectors/ft
2019-08-19 20:47:21,062 : MainThread : INFO : loading pre-existing ngrams from

time: 13.1 s


In [22]:
models, results = {}, {}

for year in tasks:
    models[f"CBOW-W2V-{year}"] = Average(w2v, wv_mapfile_path = "data/vectors/w2v")
    models[f"SIF-W2V-{year}"] = SIF(w2v, wv_mapfile_path = "data/vectors/w2v")
    
    models[f"CBOW-FT-{year}"] = Average(ft, wv_mapfile_path = "data/vectors/ft")
    models[f"SIF-FT-{year}"] = SIF(ft, wv_mapfile_path = "data/vectors/ft")
    
for k in models:
    results[k] = 0

2019-08-19 20:47:33,785 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/w2v
2019-08-19 20:47:33,807 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/w2v
2019-08-19 20:47:33,854 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/ft
2019-08-19 20:47:33,874 : MainThread : INFO : loading pre-existing vocab from data/vectors/ft
2019-08-19 20:47:33,893 : MainThread : INFO : loading pre-existing ngrams from data/vectors/ft
2019-08-19 20:47:33,927 : MainThread : INFO : loading pre-existing wv from /Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/data/vectors/ft
2019-08-19 20

time: 1.16 s


In [23]:
for year in tasks:
    df = sts_data[sts_data.task == year]
    sents_a = df.sent_a.values.tolist()
    sents_b = df.sent_b.values.tolist()
    assert len(sents_a) == len(sents_b)

    task_length = len(sents_a)
    sents_a.extend(sents_b)
    
    sents = [IndexedSentence(s, i) for i,s in enumerate(sents_a)]
    
    for k, m in models.items():
        if k.split("-")[-1] != year:
            continue
        
        m_type  = k.split("-")[0]
        emb_type = k.split("-")[1]
        
        m.train(sents)
        r = pearsonr(df.sim, compute_similarities(task_length, m))[0].round(4)
        
        results[f"{m_type}-{emb_type}-{year}"] = r * 100
        print(k, r)

2019-08-19 20:49:04,859 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 20:49:04,863 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 total words
2019-08-19 20:49:06,113 : MainThread : INFO : estimated memory for 504 sentences with 300 dimensions and 3000000 vocabulary: 3445 MB (3 GB)
2019-08-19 20:49:06,114 : MainThread : INFO : initializing sentence vectors for 504 sentences
2019-08-19 20:49:06,174 : MainThread : INFO : begin training
2019-08-19 20:49:06,397 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 20:49:06,399 : MainThread : INFO : training on 504 effective sentences with 5060 effective words took 0s with 2236 sentences/s
2019-08-19 20:49:06,446 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 20:49:06,448 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 total words


CBOW-W2V-2012 0.7467


2019-08-19 20:49:07,700 : MainThread : INFO : estimated memory for 504 sentences with 300 dimensions and 3000000 vocabulary: 3445 MB (3 GB)
2019-08-19 20:49:07,701 : MainThread : INFO : initializing sentence vectors for 504 sentences
2019-08-19 20:49:07,703 : MainThread : INFO : pre-computing SIF weights for 3000000 words
2019-08-19 20:49:11,334 : MainThread : INFO : begin training
2019-08-19 20:49:11,368 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 20:49:11,369 : MainThread : INFO : computing 1 principal components
2019-08-19 20:49:11,406 : MainThread : INFO : removing 1 principal components
2019-08-19 20:49:11,408 : MainThread : INFO : training on 504 effective sentences with 5060 effective words took 0s with 14249 sentences/s
2019-08-19 20:49:11,416 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 20:49:11,419 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 

SIF-W2V-2012 0.7874


2019-08-19 20:49:12,186 : MainThread : WARNING : Fasttext memory estimation not yet completed
2019-08-19 20:49:12,187 : MainThread : INFO : estimated memory for 504 sentences with 300 dimensions and 2000000 vocabulary: 2297 MB (2 GB)
2019-08-19 20:49:12,187 : MainThread : INFO : initializing sentence vectors for 504 sentences
2019-08-19 20:49:12,228 : MainThread : INFO : begin training
2019-08-19 20:49:12,830 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 20:49:12,831 : MainThread : INFO : training on 504 effective sentences with 6848 effective words took 0s with 836 sentences/s
2019-08-19 20:49:12,838 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 20:49:12,840 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 total words


...

SIF-W2V-2017 0.7708


2019-08-19 20:50:06,491 : MainThread : WARNING : Fasttext memory estimation not yet completed
2019-08-19 20:50:06,491 : MainThread : INFO : estimated memory for 250 sentences with 300 dimensions and 2000000 vocabulary: 2296 MB (2 GB)
2019-08-19 20:50:06,492 : MainThread : INFO : initializing sentence vectors for 250 sentences
2019-08-19 20:50:06,528 : MainThread : INFO : begin training
2019-08-19 20:50:06,571 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 20:50:06,572 : MainThread : INFO : training on 250 effective sentences with 2319 effective words took 0s with 5679 sentences/s
2019-08-19 20:50:06,576 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 20:50:06,577 : MainThread : INFO : finished scanning 250 sentences with an average length of 9 and 2319 total words


CBOW-FT-2017 0.5091


2019-08-19 20:50:07,437 : MainThread : WARNING : Fasttext memory estimation not yet completed
2019-08-19 20:50:07,438 : MainThread : INFO : estimated memory for 250 sentences with 300 dimensions and 2000000 vocabulary: 2296 MB (2 GB)
2019-08-19 20:50:07,438 : MainThread : INFO : initializing sentence vectors for 250 sentences
2019-08-19 20:50:07,440 : MainThread : INFO : pre-computing SIF weights for 2000000 words
2019-08-19 20:50:09,896 : MainThread : INFO : begin training
2019-08-19 20:50:09,926 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 20:50:09,927 : MainThread : INFO : computing 1 principal components
2019-08-19 20:50:09,932 : MainThread : INFO : removing 1 principal components
2019-08-19 20:50:09,933 : MainThread : INFO : training on 250 effective sentences with 2319 effective words took 0s with 8113 sentences/s


SIF-FT-2017 0.7487
time: 1min 5s


In [24]:
pd.DataFrame.from_dict(results, orient="index", columns=["Pearson"])

,Pearson
CBOW-W2V-2012,74.67
SIF-W2V-2012,78.74
CBOW-FT-2012,50.99
SIF-FT-2012,79.10
CBOW-W2V-2013,64.74
SIF-W2V-2013,67.41
CBOW-FT-2013,65.06
SIF-FT-2013,70.39
CBOW-W2V-2014,71.05
SIF-W2V-2014,75.50


time: 22.3 ms
